In [1]:
%tensorflow_version 2.x

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [3]:
#USE THIS CELL ONLY IF USING GOOGLE COLABORATORY
#Mounting google drive to import data into colaboratory

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#USE THIS CELL ONLY IF USING GOOGLE COLABORATORY
!unzip -uq "RAW_INPUT_DATASET_ZIP_FILE_PATH_IN_GOOGLE_DRIVE" -d "PATH_TO_DIRECTORY_WHERE_DATA_UNZIPS"

In [5]:
import os

In [6]:
def display_prediction_stats(probability_of_crack):
  print("Probability that the sample has a Crack =", probability_of_crack)
  if probability_of_crack >= 0.5:
    print("Crack Detected !")
  else:
    print("No Crack Detected !")


In [ ]:
# Joining Directory Paths

for root,directory_names,file_names in os.walk('INPUT_DATA_DIRECTORY_PATH', topdown=True):
  for each_file_name in file_names:
    os.path.join(root, each_file_name)

In [8]:
# Data Preprocessing

image_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                     validation_split=0.2,
                                     horizontal_flip=True, #Data Augmentation
                                     vertical_flip=True,   #Data Augmentation
                                     rotation_range=15,    #Data Augmentation
                                     fill_mode='nearest')

training_dataset = image_generator.flow_from_directory('input',
                                                       batch_size=32,
                                                       shuffle=True,
                                                       target_size=(128,128),
                                                       subset="training",
                                                       class_mode="binary")

validation_dataset = image_generator.flow_from_directory('input',
                                                       batch_size=32,
                                                       shuffle=True,
                                                       target_size=(128,128),
                                                       subset="validation",
                                                       class_mode="binary")

# Original Dataset divided into Training(80%) and Validation(20%) datasets

Found 32000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.


In [9]:
from tensorflow.keras import layers,models

crack_detection_model = models.Sequential()

# Input Layer - Convolutional
crack_detection_model.add(layers.Conv2D(256, # 256 - no of filters (also, no of output images)
                                 (1,1),
                                 strides=(1,1),
                                 padding="valid",
                                 use_bias=True,
                                 input_shape=(128,128,3),
                                 activation="relu"
                                 )
                          )

# Five Hidden layers - Maxpool, Convolutional, Maxpool, Convolutional, Maxpool
crack_detection_model.add(layers.MaxPooling2D(pool_size=(2,2),
                                       padding="valid",
                                       strides=(2,2), # In MaxPooling2D, strides anyway default to pool_size
                                       )
                         )

crack_detection_model.add(layers.Conv2D(256, #no of filters (also, no of output images)
                                 (1,1),
                                 strides=(1,1),
                                 padding="valid",
                                 use_bias=True,
                                 activation="relu"
                                 )
                          )

crack_detection_model.add(layers.MaxPooling2D(pool_size=(2,2),
                                       padding="valid",
                                       strides=(2,2),
                                       )
                         )

crack_detection_model.add(layers.Conv2D(256,
                                 (1,1),
                                 strides=(1,1),
                                 padding="valid",
                                 use_bias=True,
                                 activation="relu"
                                 )
                          )

crack_detection_model.add(layers.MaxPooling2D(pool_size=(2,2),
                                       padding="valid",
                                       strides=(1,1)
                                       )
                         )

 #Flattening the pixel matrix to feed the dense layer
crack_detection_model.add(layers.Flatten())

# Three Hidden Layers - Dense NN
crack_detection_model.add(layers.Dense(128,
                                       activation='relu',
                                       kernel_regularizer='l2'),
                          )

crack_detection_model.add(layers.Dense(64,
                                       activation='relu',
                                       kernel_regularizer='l2'),
                          )

crack_detection_model.add(layers.Dense(32,
                                       activation='relu',
                                       kernel_regularizer='l2'),
                          )

crack_detection_model.add(layers.Dense(8,
                                       activation='relu',
                                       kernel_regularizer='l2'),
                          )

# Output Layer - Single node with Sigmoid/SoftMax activation function for classification
crack_detection_model.add(layers.Dense(1,
                                       activation='sigmoid',
                                       kernel_regularizer='l2'),
                          )

In [10]:
crack_detection_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 256)     1024      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 256)       65792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 256)       65792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 246016)            0

In [11]:
crack_detection_model.compile(loss='binary_crossentropy', 
                              optimizer=keras.optimizers.Adam(), 
                              metrics=['accuracy'],
                              )

In [12]:
hist = crack_detection_model.fit(training_dataset,
                                 validation_data=validation_dataset, 
                                 epochs=10,
                                 batch_size=128,
                                 )

Epoch 1/10
1000/1000 [==============================] - 3070s 3s/step - loss: 0.5396 - accuracy: 0.9166 - val_loss: 0.2541 - val_accuracy: 0.9801
Epoch 2/10
1000/1000 [==============================] - 3050s 3s/step - loss: 0.2536 - accuracy: 0.9737 - val_loss: 0.2043 - val_accuracy: 0.9795
Epoch 3/10
1000/1000 [==============================] - 3056s 3s/step - loss: 0.2110 - accuracy: 0.9748 - val_loss: 0.1808 - val_accuracy: 0.9691
Epoch 4/10
1000/1000 [==============================] - 3035s 3s/step - loss: 0.1784 - accuracy: 0.9768 - val_loss: 0.2002 - val_accuracy: 0.9803
Epoch 5/10
1000/1000 [==============================] - 3040s 3s/step - loss: 0.1642 - accuracy: 0.9773 - val_loss: 0.1448 - val_accuracy: 0.9781
Epoch 6/10
1000/1000 [==============================] - 3034s 3s/step - loss: 0.1417 - accuracy: 0.9778 - val_loss: 0.1138 - val_accuracy: 0.9827
Epoch 7/10
1000/1000 [==============================] - 3028s 3s/step - loss: 0.1407 - accuracy: 0.9781 - val_loss: 0.1130 -

In [13]:
# We have trained our prediction model with almost 98% accuracy on the validiation dataset

#Let us test our model on some randomly selected images

In [14]:
img_path1 = "/content/cracked.jpg"

img1 = keras.preprocessing.image.load_img(img_path1, target_size=(128, 128))
img_array1 = keras.preprocessing.image.img_to_array(img1)/255

img_batch1 = np.expand_dims(img_array1,[0])

In [15]:
prediction1 = crack_detection_model.predict(img_batch1)

print(prediction1[0][0])
display_prediction_stats(prediction1[0][0])

0.8033247
Probability that the sample has a Crack = 0.8033247
Crack Detected !


In [16]:
img_path2 = "/content/no_crack.jpg"

img2 = keras.preprocessing.image.load_img(img_path2, target_size=(128, 128))
img_array2 = keras.preprocessing.image.img_to_array(img2)/255

img_batch2 = np.expand_dims(img_array2,[0])


In [17]:
prediction2 = crack_detection_model.predict(img_batch2)
display_prediction_stats(prediction2[0][0])

Probability that the sample has a Crack = 0.011546552
No Crack Detected !


In [18]:
img_path3 = "/content/no_crack2.jpg"

img3 = keras.preprocessing.image.load_img(img_path3, target_size=(128, 128))
img_array3 = keras.preprocessing.image.img_to_array(img3)/255

img_batch3 = np.expand_dims(img_array3,[0])

In [19]:
prediction3 = crack_detection_model.predict(img_batch3)
display_prediction_stats(prediction3[0][0])

Probability that the sample has a Crack = 0.0041581094
No Crack Detected !
